#### Purpose: Derive a measure for when a flood event is sufficiently captured 

Method:

* metric based on overlap between flood extent and the ground-level swath
* we use the "point in polygon" analysis available from shapely (the same thing used by Geopandas)

In [5]:
import pandas as pd
import pickle
import rasterio
import numpy as np
import shapely
tif_root = "STEP 1 - Data Acquisition/Global Flood Database/TIF/unzipped/"

In [6]:
# dataframe of daily flood images and sentinel coverage as bbox data
with open('flooddays_with_sentinel_df.pkl', 'rb') as f:
    flooddays_with_sentinel_df = pickle.load(f) 
f.close()

flooddays_with_sentinel_df.head()

,DFO_id,flood_day,tif_filename,flood_year,flood_start,reported_duration,observed_total_duration,snapshot_date,snapshot_extent_img,snapshot_extent_km2,...,displaced_k,duration_days,exposed_mn,killed,start_date,end_date,dfo_severity,wsg84_bbox,sentinel_coverage,sentinel_coverage_Nboxes
DFO_day_id,,,,,,,,,,,,,,,,,,,,,
DFO_4632_0,DFO_4632,0,DFO_4632_From_20180615_to_20180620.tif,2018,2018-06-15,5,8,2018-06-15,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",7513.3125,...,1000,5,1207989,1,2018-06-15,2018-06-20,1.5,"[90.63215190892367, 32.771664458785295, 108.43...","{0: [91.9456071436343, 29.77601126852889, 94.8...",5
DFO_4632_1,DFO_4632,1,DFO_4632_From_20180615_to_20180620.tif,2018,2018-06-15,5,8,2018-06-16,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5362.2500,...,1000,5,1207989,1,2018-06-15,2018-06-20,1.5,"[90.63215190892367, 32.771664458785295, 108.43...",{},0
DFO_4632_2,DFO_4632,2,DFO_4632_From_20180615_to_20180620.tif,2018,2018-06-15,5,8,2018-06-17,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2904.8125,...,1000,5,1207989,1,2018-06-15,2018-06-20,1.5,"[90.63215190892367, 32.771664458785295, 108.43...","{0: [95.9592518987471, 29.26117989441572, 98.8...",5
DFO_4632_3,DFO_4632,3,DFO_4632_From_20180615_to_20180620.tif,2018,2018-06-15,5,8,2018-06-18,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1152.7500,...,1000,5,1207989,1,2018-06-15,2018-06-20,1.5,"[90.63215190892367, 32.771664458785295, 108.43...",{},0
DFO_4632_4,DFO_4632,4,DFO_4632_From_20180615_to_20180620.tif,2018,2018-06-15,5,8,2018-06-19,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",652.3750,...,1000,5,1207989,1,2018-06-15,2018-06-20,1.5,"[90.63215190892367, 32.771664458785295, 108.43...","{0: [100.28209710959871, 30.251160866016946, 1...",5


[I] function to create a list of wgs84 coordinates for each flood pixel in a snapshot image

In [7]:
def wgs_flood_coordlist_from_img(img, transfm):
    """
    given a binary image and the parameters for an (affine) transformation, this function will
    return the WGS684 picel coordinates of all the positive pixels
    """
    indices = np.where(img >0)
    coordinates = zip(indices[0], indices[1])

    wgs_list = []
    for xy in list(coordinates):
        (wgs_x, wgs_y) = rasterio.transform.xy(transfm,xy[0],xy[1])
        wgs_list.append((wgs_x, wgs_y))
    
    return wgs_list



... test that we get a list of believable coordinates from a single flood extent image

In [8]:
gfd_root  = 0 #arbitrary image

# load the test image with rasterio to obtain the projection information
img_filename = tif_root + flooddays_with_sentinel_df.tif_filename.iloc[gfd_root]
raster=rasterio.open(img_filename)
spatial_transform = raster.meta['transform']

# get the corresponding image that is stored in the dataframe
test_image = flooddays_with_sentinel_df.snapshot_extent_img.iloc[gfd_root]

test_coords = wgs_flood_coordlist_from_img(test_image, spatial_transform)

test_coords

[(92.4624693003172, 32.744715000261714),
 (92.13233843340328, 32.733486059210215),
 (92.52535137020557, 32.733486059210215),
 (92.52759715841586, 32.733486059210215),
 (92.53208873483646, 32.72674869457932),
 (92.53658031125705, 32.72674869457932),
 (93.0149332000507, 32.72674869457932),
 (92.73420967376336, 32.72001132994843),
 (93.0014584707889, 32.717765541738125),
 (92.0829310927767, 32.702045024266035),
 (92.0784395163561, 32.697553447845436),
 (92.0896684574076, 32.697553447845436),
 (92.0919142456179, 32.697553447845436),
 (92.0919142456179, 32.69530765963514),
 (92.8487448724886, 32.69306187142484),
 (92.13458422161358, 32.69081608321454),
 (92.11437212772088, 32.68857029500424),
 (92.11661791593119, 32.68857029500424),
 (92.11886370414148, 32.68857029500424),
 (92.04924426962222, 32.686324506793945),
 (93.0194247764713, 32.675095565742446),
 (92.12784685698267, 32.670603989321854),
 (92.13009264519297, 32.670603989321854),
 (92.13233843340328, 32.670603989321854),
 (92.1345842

[II] function to count how many of the flood pixels are inside the bounding box of the Sentinel ground-level coverage

In [9]:
def capture_metrics(list_of_coords, bbox):
    """
    Given a list of coordinates in the wgs84 stystem, and the 4 corners of a bounding box, this function returns
    a count and the percentage of the coords in the list, which fall inside the bounding box
    """
    polygon = shapely.geometry.Polygon(bbox)

    count=0
    for wgs_point in list_of_coords:
        point = shapely.geometry.Point(wgs_point)
        count+=point.within(polygon)
    
    return count, count/len(list_of_coords)


... test that we get expected results for a simple case: 1 out of 4 pixel locations known to fall inside the bounding box

In [13]:
bbox = [
        (24.950899, 60.169158),
        (24.953492, 60.169158),
        (24.953510, 60.170104),
        (24.950958, 60.169990)
    ]

list_of_coords = [(24.952242, 60.1696017), # this one known to be in box
                (74.20196536237763, 30.775158739829656),
                (74.20421115058794, 30.775158739829656),
                (74.20645693879823, 30.775158739829656),
                ]

test_count, test_percent = capture_metrics(list_of_coords, bbox)

print(test_count, "pixels withion Sentinel ground coverage")
print(test_percent, "= proportion of flood extent captured")

1 pixels withion Sentinel ground coverage
0.25 = proportion of flood extent captured
